In [5]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd


df = pd.read_csv(r"C:\Users\yerra\Desktop\projects\dataset.csv")


df.ffill(inplace=True)


app = Dash(__name__)


app.layout = html.Div([
    html.H1("AWS Cloud-Based Data-Driven Performance and Efficiency Dashboard 2024", style={'text-align': 'center'}),
    
    
    dcc.Dropdown(
        id='category-dropdown',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'SUVs', 'value': 'SUVs'},
            {'label': 'Electric Vehicles', 'value': 'Electric Vehicles'}
        ],
        value='All',
        style={'width': '48%', 'display': 'inline-block', 'margin-bottom': '10px'}
    ),
    
    
    dcc.Dropdown(
        id='metric-dropdown-1',
        options=[
            {'label': 'Fuel Consumption (l/100km)', 'value': 'Fuel Consumption (l/100km)'},
            {'label': 'Energy Consumption in (kWh/100km)', 'value': 'Energy Consumption in (kWh/100km)'},
            {'label': 'Weighted Combined in PHEV  (kWh/100km)', 'value': 'Weighted Combined in PHEV  (kWh/100km)'},
            {'label': 'CO2 Emissions (g/km)', 'value': 'CO2 Emissions (g/km)'}
        ],
        value='Fuel Consumption (l/100km)',
        style={'width': '48%', 'display': 'inline-block', 'margin-right': '2%'}
    ),
    
    dcc.Dropdown(
        id='metric-dropdown-2',
        options=[
            {'label': 'Fuel Consumption (l/100km)', 'value': 'Fuel Consumption (l/100km)'},
            {'label': 'Energy Consumption in (kWh/100km)', 'value': 'Energy Consumption in (kWh/100km)'},
            {'label': 'Weighted Combined in PHEV  (kWh/100km)', 'value': 'Weighted Combined in PHEV  (kWh/100km)'},
            {'label': 'CO2 Emissions (g/km)', 'value': 'CO2 Emissions (g/km)'}
        ],
        value='CO2 Emissions (g/km)',
        style={'width': '48%', 'display': 'inline-block'}
    ),
    
    # Additional graphs
    html.Div([
        dcc.Graph(id='comparison-chart-1', style={'width': '48%', 'display': 'inline-block'}),
        dcc.Graph(id='comparison-chart-2', style={'width': '48%', 'display': 'inline-block'}),
        dcc.Graph(id='boxplot', style={'width': '48%', 'display': 'inline-block'}),
        dcc.Graph(id='histogram', style={'width': '48%', 'display': 'inline-block'}),
        dcc.Graph(id='heatmap', style={'width': '48%', 'display': 'inline-block'})
    ]),
])




@app.callback(
    [Output('comparison-chart-1', 'figure'), Output('comparison-chart-2', 'figure')],
    [Input('metric-dropdown-1', 'value'), Input('metric-dropdown-2', 'value'), Input('category-dropdown', 'value')]
)
def update_comparison_charts(metric1, metric2, category):
    
    if category != 'All':
        filtered_df = df[df['Category'] == category]
    else:
        filtered_df = df

    # Convert to numeric and drop NaNs for valid comparison
    filtered_df[metric1] = pd.to_numeric(filtered_df[metric1], errors='coerce')
    filtered_df[metric2] = pd.to_numeric(filtered_df[metric2], errors='coerce')
    filtered_df.dropna(subset=[metric1, metric2], inplace=True)

    
    fig1 = px.line(filtered_df, x='Model', y=metric1, color='Category', title=f"Comparison of {metric1}", markers=True)
    fig1.update_layout(
        xaxis_title='Model',
        yaxis_title=metric1,
        xaxis_tickangle=-45,
        showlegend=True,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        yaxis=dict(showgrid=True, gridcolor='lightgray')
    )

    
    fig2 = px.line(filtered_df, x='Model', y=metric2, color='Category', title=f"Comparison of {metric2}", markers=True)
    fig2.update_layout(
        xaxis_title='Model',
        yaxis_title=metric2,
        xaxis_tickangle=-45,
        showlegend=True,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        yaxis=dict(showgrid=True, gridcolor='lightgray')
    )

    return fig1, fig2


@app.callback(
    Output('boxplot', 'figure'),
    Input('metric-dropdown-1', 'value')
)
def update_boxplot(metric):
    fig = px.box(df, x='Category', y=metric, title=f'Box Plot of {metric} by Category')
    fig.update_layout(xaxis_title='Category', yaxis_title=metric, plot_bgcolor='rgba(0, 0, 0, 0)')
    return fig


@app.callback(
    Output('histogram', 'figure'),
    Input('metric-dropdown-1', 'value')
)
def update_histogram(metric):
    fig = px.histogram(df, x=metric, title=f'Histogram of {metric}')
    fig.update_layout(xaxis_title=metric, yaxis_title='Count', plot_bgcolor='rgba(0, 0, 0, 0)')
    return fig


@app.callback(
    Output('heatmap', 'figure'),
    Input('category-dropdown', 'value')
)
def update_heatmap(category):
    if category != 'All':
        filtered_df = df[df['Category'] == category]
    else:
        filtered_df = df

    
    numeric_columns = ['Energy Consumption in (kWh/100km)', 'Weighted Combined in PHEV  (kWh/100km)', 
                       'CO2 Emissions (g/km)', 'Fuel Consumption (l/100km)']
    
    filtered_df = filtered_df[numeric_columns].apply(pd.to_numeric, errors='coerce').dropna()
    
    if not filtered_df.empty:
        corr_matrix = filtered_df.corr()
        fig = px.imshow(corr_matrix, text_auto=True, title=f'Correlation Heatmap for {category} Category')
        fig.update_layout(xaxis_title='Metrics', yaxis_title='Metrics', plot_bgcolor='rgba(0, 0, 0, 0)')
        return fig
    return {}


if __name__ == '__main__':
    app.run_server(debug=True, port=8052)
